<a href="https://colab.research.google.com/github/MarinaaaniraM/deep_test/blob/master/cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation
from keras.layers import Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import SGD

Using TensorFlow backend.


In [0]:
# Задаем seed для повторяемости результатов
numpy.random.seed(42)

# Загружаем данные
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
# Размер мини-выборки
batch_size = 32
# Количество классов изображений
nb_classes = 10
# Количество эпох для обучения
nb_epoch = 25
# Размер изображений
img_rows, img_cols = 32, 32
# Количество каналов в изображении: RGB
img_channels = 3

170500096/170498071 [==============================] - 29s 0us/step


In [0]:
# Нормализуем данные
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# Преобразуем метки в категории
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [0]:
# Создаем последовательную модель
model = Sequential()
# Первый сверточный слой
model.add(Conv2D(32, (3, 3), padding='same',
                        input_shape=(32, 32, 3), activation='relu'))
# Второй сверточный слой
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
# Первый слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))
# Слой регуляризации Dropout
model.add(Dropout(0.25))

# Третий сверточный слой
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
# Четвертый сверточный слой
model.add(Conv2D(64, (3, 3), activation='relu'))
# Второй слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))
# Слой регуляризации Dropout
model.add(Dropout(0.25))
# Слой преобразования данных из 2D представления в плоское
model.add(Flatten())
# Полносвязный слой для классификации
model.add(Dense(512, activation='relu'))
# Слой регуляризации Dropout
model.add(Dropout(0.5))
# Выходной полносвязный слой
model.add(Dense(nb_classes, activation='softmax'))

In [0]:
# Задаем параметры оптимизации
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [0]:
# Обучаем модель
model.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=nb_epoch,
              validation_split=0.1,
              shuffle=True,
              verbose=2)

Train on 45000 samples, validate on 5000 samples
Epoch 1/25
 - 270s - loss: 1.7743 - acc: 0.3470 - val_loss: 1.3388 - val_acc: 0.5144
Epoch 2/25
 - 269s - loss: 1.3174 - acc: 0.5258 - val_loss: 1.1457 - val_acc: 0.5910
Epoch 3/25
 - 271s - loss: 1.1511 - acc: 0.5901 - val_loss: 1.0422 - val_acc: 0.6294
Epoch 4/25
 - 270s - loss: 1.0470 - acc: 0.6298 - val_loss: 0.8911 - val_acc: 0.6814
Epoch 5/25
 - 269s - loss: 0.9739 - acc: 0.6560 - val_loss: 0.8673 - val_acc: 0.6986
Epoch 6/25
 - 268s - loss: 0.9094 - acc: 0.6794 - val_loss: 0.7922 - val_acc: 0.7238
Epoch 7/25
 - 268s - loss: 0.8656 - acc: 0.6941 - val_loss: 0.7999 - val_acc: 0.7150
Epoch 8/25
 - 270s - loss: 0.8265 - acc: 0.7088 - val_loss: 0.8159 - val_acc: 0.7244
Epoch 9/25
 - 270s - loss: 0.8013 - acc: 0.7198 - val_loss: 0.7887 - val_acc: 0.7254
Epoch 10/25
 - 268s - loss: 0.7731 - acc: 0.7312 - val_loss: 0.8090 - val_acc: 0.7310
Epoch 11/25
 - 269s - loss: 0.7553 - acc: 0.7381 - val_loss: 0.7406 - val_acc: 0.7488
Epoch 12/25
 -

In [0]:
# Оцениваем качество обучения модели на тестовых данных
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Точность работы на тестовых данных: %.2f%%" % (scores[1]*100))

Точность работы на тестовых данных: 74.73%


In [0]:
print("Сохраняем сеть")
# Сохраняем сеть для последующего использования
# Генерируем описание модели в формате json
model_json = model.to_json()
json_file = open("cifar10_model.json", "w")
# Записываем архитектуру сети в файл
json_file.write(model_json)
json_file.close()
# Записываем данные о весах в файл
model.save_weights("cifar10_model.h5")
print("Сохранение сети завершено")